In [ ]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import TensorBoard
from vit_keras import vit

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Khai báo các thông số
img_width, img_height = 224, 224
batch_size = 16
epochs = 20
num_classes = 3

In [ ]:
# Đường dẫn đến thư mục chứa dữ liệu
data_dir = '../dataset/0.5'

# Danh sách các lớp (classes) là các thư mục con trong thư mục chứa dữ liệu
classes = os.listdir(data_dir)
print(classes)

In [ ]:
# Tạo empty lists để lưu trữ đường dẫn hình ảnh và nhãn tương ứng
image_paths = []
labels = []

# Lặp qua mỗi lớp
for class_label in classes:
  class_dir = os.path.join(data_dir, class_label)
  # Lặp qua mỗi hình ảnh trong mỗi lớp
  for image_name in os.listdir(class_dir):
    image_path = os.path.join(class_dir, image_name)
    image_paths.append(image_path)
    labels.append(class_label)

# Chia dữ liệu thành tập huấn luyện và tập validation
train_image_paths, val_image_paths, train_labels, val_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

# Tạo DataFrame cho tập huấn luyện
train_dataframe = pd.DataFrame({'image_path': train_image_paths, 'label': train_labels})
# Tạo DataFrame cho tập validation
val_dataframe = pd.DataFrame({'image_path': val_image_paths, 'label': val_labels})

In [ ]:
# Tạo ImageDataGenerator cho tập huấn luyện
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Chuẩn hóa lại giá trị pixel về khoảng [0,1]
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Tạo ImageDataGenerator cho tập validation, chỉ cần rescale lại giá trị pixel
val_datagen = ImageDataGenerator(rescale=1./255)

# Tạo generator cho tập huấn luyện
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_dataframe,  # DataFrame chứa đường dẫn hình ảnh và nhãn cho tập huấn luyện
    x_col="image_path",
    y_col="label",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# Tạo generator cho tập validation
validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_dataframe,  # DataFrame chứa đường dẫn hình ảnh và nhãn cho tập validation
    x_col="image_path",
    y_col="label",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
validation_generator[1][0].shape

In [ ]:
# Tạo mô hình Vision Transformer
model = vit.vit_b16(
    image_size=img_width,
    activation='softmax',
    pretrained=True,
    include_top=True,
    pretrained_top=False,
    classes=num_classes,
)

In [ ]:
# Compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
# Định nghĩa callback để ghi log TensorBoard
log_dir = "/content/drive/MyDrive/Colab Notebooks/Experimental/ViT/logs/fit"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples // batch_size)

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print('Test accuracy:', test_acc)

In [ ]:
image_path = '/content/drive/MyDrive/Colab Notebooks/Experimental/test_image.jpg'
img = load_img(image_path, target_size=(img_width, img_height))
img_array = img_to_array(img)

# Rescale hình ảnh
img_array /= 255.
img_array = img_array.reshape(1, 224, 224, 3)
img_array.shape

In [ ]:
model.predict(img_array)

In [ ]:
# Visualize loss and accuracy
import matplotlib.pyplot as plt

train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs_range = range(1, epochs+1)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, train_loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, train_acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.show()